## 文本索引与答案检索

### 任务

- 任务说明：文本文本索引的实现逻辑
- 任务要求：
  - 理解倒排索引
  - 实现TFIDF和BM25的编码与检索
- 打卡要求：使用TFIDF和BM25进行检索，使用question检索到答案的reference页面位置

### 文本检索流程

- 文本预处理阶段
  - 对原始文本进行清理和规范化，包括去除停用词、标点符号等噪声，并将文本统一转为小写。
  - 采用词干化或词形还原等技术，将单词转换为基本形式，以减少词汇的多样性，为后续建立索引做准备。

- 构建倒排索引
  - **对文档集合进行分词**，得到每个文档的词项列表，
  - **为每个词项构建倒排列表**
  - **记录包含该词项的文档及其位置信息**。这种结构使得在查询时能够快速找到包含查询词的文档，为后续的文本检索奠定了基础
- 文本检索
  - 对用户查询进行预处理
  - 处理后的用户查询与建立的倒排索引进行匹配
  - 计算查询中每个词项的权重，并利用检索算法（如TFIDF或BM25）对文档进行排序，将相关性较高的文档排在前面。

### 文本检索与语义检索

|              | 文本检索                                                     | 语义检索                                             |
| :----------- | :----------------------------------------------------------- | ---------------------------------------------------- |
| **定义**     | 通过关键词或短语匹配文本数据的过程                           | 强调理解查询与文本之间的深层语义关系                 |
| **方法**     | 基于关键词匹配，使用TFIDF、BM25等权重计算                    | 使用NLP技术，如词嵌入、预训练的语言模型              |
| **特点**     | 强调**字面意义**，关注表面文本的匹配                         | 关注词语之间的关联、**语境和含义**                   |
| **应用场景** | 大规模文本数据的快速匹配                                     | 对语义理解要求较高的场景                             |
| **优势**     | 处理速度较快，适用于大规模文本数据                           | 能够处理一词多义、近义词等语义上的复杂情况           |
| **联系**     | 结合使用，先使用文本检索筛选出候选文档，然后在这些文档上应用语义检索 | 可以利用语义模型提取关键词的上下文信息，提升检索效果 |

一般使用步骤：

- 先使用文本检索筛选出候选文档

- 然后在这些文档上应用语义检索来进一步提高检索的准确性

- 在RAG中可以结合两种方法一起进行使用。

#### TFIDF

TFIDF是一种用于**信息检索和文本挖掘的常用权重计算方法**，旨在衡量**一个词项对于一个文档集合中某个文档的重要性**。该方法结合了两个方面的信息：词项在文档中的频率（TF）和在整个文档集合中的逆文档频率（IDF）。

##### **词项在文档中的频率（TF）**：

TF表示一个**词在文档中出现的次数相对于文档总词数的比例**。
$$
TF(t,d) = \frac{词项t在文档d中出现的次数}{文档d中所有词的总数}
$$

##### **逆文档频率（IDF）**

IDF表示了**一个词项在整个文档集合中的稀有程度**，如果词项在许多文档中都出现，其IDF值较低，反之则较高。
$$
IDF(t) = log(\frac{文档集合中的文档总数}{包含词项t的文档数 + 1})
$$

##### **TFIDF的计算**

TFIDF的最终值是将词项在文档中的频率和在整个文档集合中的逆文档频率相乘 
$$
TFIDF(t,d,D) = TF(t,d)*IDF(t)
$$

#### BM25

BM25Okapi是BM25算法的一种变体，它在信息检索中用于评估文档与查询之间的相关性。以下是BM25Okapi的原理和打分方式的概述：

- BM25Okapi的主要参数：

  - $k1$：控制词项频率对分数的影响，通常设置为1.5。

  - $b$：控制文档长度对分数的影响，通常设置为0.75。

  - $epsilon$：用于防止逆文档频率（IDF）为负值的情况，通常设置为0.25。

- 打分的计算过程：

  - TF（词项在文档中的频率）

  - IDF（逆文档频率）

  - 文档长度（docs_len）
  - 文档长度对分数的影响通过 $b$ 控制。文档长度越长，该项的分数越小。BM25Okapi的打分公式综合考虑了以上三个因素，通过对每个词项的打分求和得到最终的文档与查询的相关性分数。

$$
score = \sum_{q_{query}}(IDF(q)*\frac{q_{r}*req*(k1+1)}{q_{r}*req+k1(1-b+b*\frac{docs\_len}{avg\_doc\_len})})
$$

- 
  - ${avg\_doc\_len}$是文档集合中的平均文档长度。BM25Okapi通过合理调整参数，兼顾了词项频率、文档长度和逆文档频率，使得在信息检索任务中能够更准确地评估文档与查询之间的相关性，提高检索效果。

### 文本切分方法

文本的长度是另一个关键因素，影响了文本编码的结果。短文本和长文本在编码成向量时可能表达不同的语义信息。即使两者包含相同的单词或有相似的语义，由于上下文的不同，得到的向量也会有所不同。因此，当在语义检索中使用短文本来检索长文本时，或者反之，可能导致一定的误差。针对文本长度的差异，有些系统采用截断或填充等方式处理，以保持一致的向量表示。



| 名称           | 分割依据                   | 描述                                                         |
| :------------- | :------------------------- | :----------------------------------------------------------- |
| 递归式分割器   | 一组用户定义的字符         | 递归地分割文本。递归分割文本的目的是尽量保持相关的文本段落相邻。这是开始文本分割的推荐方式。 |
| HTML分割器     | HTML特定字符               | 基于HTML特定字符进行文本分割。特别地，它会添加有关每个文本块来源的相关信息（基于HTML结构）。 |
| Markdown分割器 | Markdown特定字符           | 基于Markdown特定字符进行文本分割。特别地，它会添加有关每个文本块来源的相关信息（基于Markdown结构）。 |
| 代码分割器     | 代码（Python、JS）特定字符 | 基于特定于编码语言的字符进行文本分割。支持从15种不同的编程语言中选择。 |
| Token分割器    | Tokens                     | 基于Token进行文本分割。存在一些不同的Token计量方法。         |
| 字符分割器     | 用户定义的字符             | 基于用户定义的字符进行文本分割。这是较为简单的分割方法之一。 |
| 语义分块器     | 句子                       | 首先基于句子进行分割。然后，如果它们在语义上足够相似，就将相邻的句子组合在一起。 |



对于自然语言，可以推荐使用Token分割器，结合Chunk Size和Overlap Size可以得到不同的切分：

- **Chunk Size（块大小）**：表示将文本划分为较小块的大小。这是分割后每个独立文本块的长度或容量。块大小的选择取决于应用的需求和对文本结构的理解。
- **Overlap Size（重叠大小）**：指相邻两个文本块之间的重叠部分的大小。在切割文本时，通常希望保留一些上下文信息，重叠大小就是控制这种上下文保留的参数。
- 

### 参考资料

[BM25算法, Best Matching](https://zhuanlan.zhihu.com/p/79202151)



更多阅读资料：

- https://python.langchain.com/docs/modules/data_connection/document_transformers/
- https://chunkviz.up.railway.app/


In [4]:
import spacy
import re
from tqdm import tqdm
import json
import pdfplumber
from langchain.schema import Document
import jieba
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
import pandas as pd 

page_content  = []

In [5]:

def extract_page_text(filepath, max_len=256, overlap_len=100):
    page_content  = []
    pdf =pdfplumber.open(filepath)
    page_count = 0
    # pattern = r'^\d{1,3}'
    for page in tqdm(pdf.pages):
        page_text = page.extract_text().strip()
        raw_text = [text.strip() for text in page_text.split('\n')]
        new_text = '\n'.join(raw_text)
        new_text = re.sub(r'\n\d{2,3}\s?', '\n', new_text)
        # new_text = re.sub(pattern, '', new_text).strip()
        if len(new_text)>10 and '..............' not in new_text:
            page_content.append(new_text)


    cleaned_chunks = []
    i = 0
    all_str = ''.join(page_content)
    all_str = all_str.replace('\n', '')
    while i<len(all_str):
        cur_s = all_str[i:i+max_len]
        if len(cur_s)>10:
            cleaned_chunks.append(Document(page_content=cur_s, metadata={'page':page_count+1}))
        i+=(max_len - overlap_len)

    return cleaned_chunks,page_content

In [6]:
questions = json.load(open("./汽车知识问答/questions.json"))
filepath = './汽车知识问答/初赛训练数据集.pdf'
_,pdf_content = extract_page_text(filepath, max_len=256, overlap_len=100)

100%|██████████| 354/354 [00:08<00:00, 42.89it/s]


In [10]:

# 对提问和PDF内容进行分词
question_words = [' '.join(jieba.lcut(x['question'])) for x in questions]
pdf_content_words = [' '.join(jieba.lcut(x )) for x in pdf_content]


## TFIDF

In [11]:

tfidf = TfidfVectorizer()
tfidf.fit(question_words + pdf_content_words)

TfidfVectorizer()

In [12]:
# 提取TFIDF
question_feat = tfidf.transform(question_words)
pdf_content_feat = tfidf.transform(pdf_content_words)

In [13]:
# 进行归一化
question_feat = normalize(question_feat)
pdf_content_feat = normalize(pdf_content_feat)


In [60]:
# 检索进行排序
# 1. 使用循环遍历问题特征列表`question_feat`，并获取其索引`query_idx`以及对应的特征向量`feat`。
# 2. 对每个问题特征向量`feat`，通过矩阵乘法计算其与PDF内容特征矩阵`pdf_content_feat`的相似度得分，得到一个得分向量`score`。
# 3. 将得分向量转换为数组，并取第一个元素，得到具体的得分值。
# 4. 通过对得分数组的排序，找到最大得分对应的页面索引`max_score_page_idx`，并将其加1（索引从0开始）。
# 5. 将最匹配页面的索引记录在问题对象中作为参考页面，格式为'page_' + 页面索引的字符串表示。

# 该代码实现了利用向量化计算方法对问题特征与PDF内容特征进行匹配，然后将最匹配的页面索引存储到相应的问题对象中，以便后续的参考和查找操作。
for query_idx, feat in enumerate(question_feat):
    score = feat @ pdf_content_feat.T # 计算该个问题的特征与所有pdf内容特征的相似性得分
    score = score.toarray()[0] 
    max_score_page_idx = score.argsort()[-1] + 1 # 对分数进行排序 最后一个分数最大，索引+1 为对应的page页码
    questions[query_idx]['reference'] = 'page_' + str(max_score_page_idx)

print('每一个问题的特征纬度:',feat.shape,'所有PDF内容的特征纬度:',pdf_content_feat.T.shape)
# (1, 4290)* (4290, 304) = (1, 304) 每一个问题特征与所有pdf内容特征的相似性得分

每一个问题的特征纬度: (1, 4290) 所有PDF内容的特征纬度: (4290, 304)


In [61]:
# 生成提交结果
# https://competition.coggle.club/
with open('submit.json', 'w', encoding='utf8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)

## bm25

In [66]:
from rank_bm25 import BM25Okapi

pdf_content_words = [jieba.lcut(x ) for x in pdf_content]
bm25 = BM25Okapi(pdf_content_words)

for query_idx in range(len(questions)):
    doc_scores = bm25.get_scores(jieba.lcut(questions[query_idx]["question"]))
    max_score_page_idx = doc_scores.argsort()[-1] + 1
    questions[query_idx]['reference'] = 'page_' + str(max_score_page_idx)

with open('submit.json', 'w', encoding='utf8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)

### 优化方向：

- 实际进行文本检索时，特别是在大规模数据集和生产环境中，**应该使用专业的文本检索引擎工具**，例如Elasticsearch，以确保高效、可扩展和内存友好的实现。
- **对整个文本集合构建矩阵并进行相似度计算可能导致内存占用较大**，尤其在大规模数据集情况下。建议考虑使用基于倒排索引等数据结构的文本检索引擎，以减小内存占用并提高检索效率。
- **建议进行停用词的去除**，以排除常见但无实际意义的词汇，提高检索的准确性。同时，考虑引入领域专有的单词筛选，以过滤掉与任务无关的词汇，优化检索结果。
- 实际应用中，对于PDF文档，可以考虑使用专业的PDF文本提取工具，**提取有意义的文本内容**，而不是将每一页都当做独立的文档处理。这有助于更好地利用文档内部的语义信息。